In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np

from sklearn.decomposition import PCA
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler

from sklearn.utils import resample
from sklearn.model_selection import train_test_split, StratifiedKFold,GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.tree import plot_tree
from imblearn import over_sampling

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore")

data = pd.read_csv("M:/DataSet/usaccident/real/selectData1.csv",index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212697 entries, 0 to 212696
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   TMC                212697 non-null  int64  
 1   Severity           212697 non-null  int64  
 2   Start_Lat          212697 non-null  float64
 3   Start_Lng          212697 non-null  float64
 4   Distance(mi)       212697 non-null  float64
 5   Side               212697 non-null  int64  
 6   Temperature(F)     212697 non-null  float64
 7   Wind_Chill(F)      212697 non-null  float64
 8   Humidity(%)        212697 non-null  float64
 9   Pressure(in)       212697 non-null  float64
 10  Visibility(mi)     212697 non-null  float64
 11  Wind_Speed(mph)    212697 non-null  float64
 12  Precipitation(in)  212697 non-null  float64
 13  Amenity            212697 non-null  int64  
 14  Bump               212697 non-null  int64  
 15  Crossing           212697 non-null  int64  
 16  Gi

In [2]:
Y = data['Severity']
X = data.drop(["Severity"], axis=1)

del data

X.shape, Y.shape

((212697, 39), (212697,))

data split
=

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10, stratify=Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.2,random_state=10,stratify=Y_train)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape, X_val.shape, Y_train.shape, Y_test.shape, Y_val.shape

((136125, 39), (42540, 39), (34032, 39), (136125,), (42540,), (34032,))

In [4]:
Y.value_counts()

2    174295
3     36633
4      1644
1       125
Name: Severity, dtype: int64

In [5]:
pca = PCA(n_components=0.98)
pca.fit(X_train)
print(pca.explained_variance_ratio_.sum())
print(len(pca.explained_variance_))

0.9807221890965235
11


In [6]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_val = pca.transform(X_val)

X_train.shape, X_test.shape, X_val.shape

((136125, 11), (42540, 11), (34032, 11))

OverSampling
=

In [7]:
ros = over_sampling.RandomOverSampler(random_state=12)

X_train, Y_train = ros.fit_resample(X_train, Y_train)

print(sorted(Counter(Y_train).items()))

[(1, 111548), (2, 111548), (3, 111548), (4, 111548)]


Model
=

In [8]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, Y_train)

pred_dt = dtree.predict(X_test)
y_train_pred = dtree.predict(X_train[:30000])

mat_train = confusion_matrix(Y_train[:30000],y_train_pred)
mat_dt = confusion_matrix(Y_test, pred_dt)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_dt}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_dt))

del pred_dt, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [    0 24648    17    22]
 [    0     0  5064     0]
 [    0     0     0   234]]

confusion matrix :
[[    4    20     1     0]
 [   21 29620  5047   171]
 [    2  4916  2344    65]
 [    0   180    52    97]]

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00     24687
           3       1.00      1.00      1.00      5064
           4       0.91      1.00      0.96       234

    accuracy                           1.00     30000
   macro avg       0.98      1.00      0.99     30000
weighted avg       1.00      1.00      1.00     30000

              precision    recall  f1-score   support

           1       0.15      0.16      0.15        25
           2       0.85      0.85      0.85     34859
           3       0.31      0.32      0.32      7327
           4       0.29      0.29      0.29       329

    accuracy                       

In [9]:
dt_params = {
    'criterion':['gini','entropy'],
    'max_depth':[8,10,12,14,16]
}
clf = GridSearchCV(dtree, param_grid=dt_params,cv=5,scoring='recall',n_jobs=-1)
clf.fit(X_val, Y_val)


dtree = DecisionTreeClassifier(**clf.best_params_)
dtree.fit(X_train, Y_train)

pred_dt = dtree.predict(X_test)
y_train_pred = dtree.predict(X_train[:30000])

mat_train = confusion_matrix(Y_train[:30000],y_train_pred)
mat_dt = confusion_matrix(Y_test, pred_dt)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_dt}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_dt))

del pred_dt, y_train_pred

confusion matrix :
[[   14     1     0     0]
 [ 3705  7657 11034  2291]
 [  673   906  3195   290]
 [    6    14    13   201]]

confusion matrix :
[[    8     8     9     0]
 [ 5335 10662 15719  3143]
 [  875  1443  4547   462]
 [    7    50    44   228]]

              precision    recall  f1-score   support

           1       0.00      0.93      0.01        15
           2       0.89      0.31      0.46     24687
           3       0.22      0.63      0.33      5064
           4       0.07      0.86      0.13       234

    accuracy                           0.37     30000
   macro avg       0.30      0.68      0.23     30000
weighted avg       0.77      0.37      0.44     30000

              precision    recall  f1-score   support

           1       0.00      0.32      0.00        25
           2       0.88      0.31      0.45     34859
           3       0.22      0.62      0.33      7327
           4       0.06      0.69      0.11       329

    accuracy                       

In [10]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=15)
rfc.fit(X_train,Y_train)

pred_rfc = rfc.predict(X_test)
y_train_pred = rfc.predict(X_train[:30000])

mat_rfc = confusion_matrix(Y_test, pred_rfc)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_rfc}\n")

print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_rfc))

del pred_rfc, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [  137 16747  7322   481]
 [   25   543  4406    90]
 [    0     0     0   234]]

confusion matrix :
[[    5    16     4     0]
 [  195 21835 12076   753]
 [   32  2556  4539   200]
 [    0   121    25   183]]

              precision    recall  f1-score   support

           1       0.08      1.00      0.16        15
           2       0.97      0.68      0.80     24687
           3       0.38      0.87      0.52      5064
           4       0.29      1.00      0.45       234

    accuracy                           0.71     30000
   macro avg       0.43      0.89      0.48     30000
weighted avg       0.86      0.71      0.75     30000

              precision    recall  f1-score   support

           1       0.02      0.20      0.04        25
           2       0.89      0.63      0.74     34859
           3       0.27      0.62      0.38      7327
           4       0.16      0.56      0.25       329

    accuracy                       

In [11]:
rfc_params = {
    "n_estimators" : [150, 200],
    'max_depth' : [11,15,19]
}
clf = GridSearchCV(rfc, param_grid=rfc_params, scoring='recall',cv=5,n_jobs=-1)
clf.fit(X_val, Y_val)

rfc = RandomForestClassifier(**clf.best_params_)
rfc.fit(X_train,Y_train)

pred_rfc = rfc.predict(X_test)
y_train_pred = rfc.predict(X_train[:30000])

mat_rfc = confusion_matrix(Y_test, pred_rfc)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)


print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_rfc}\n")

print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_rfc))

del pred_rfc,y_train_pred

confusion matrix :
[[   15     0     0     0]
 [  625 12800  9973  1289]
 [   91  1011  3776   186]
 [    2     1     2   229]]

confusion matrix :
[[    7    14     4     0]
 [  806 17372 14863  1818]
 [  135  1976  4890   326]
 [    2    71    20   236]]

              precision    recall  f1-score   support

           1       0.02      1.00      0.04        15
           2       0.93      0.52      0.66     24687
           3       0.27      0.75      0.40      5064
           4       0.13      0.98      0.24       234

    accuracy                           0.56     30000
   macro avg       0.34      0.81      0.34     30000
weighted avg       0.81      0.56      0.62     30000

              precision    recall  f1-score   support

           1       0.01      0.28      0.01        25
           2       0.89      0.50      0.64     34859
           3       0.25      0.67      0.36      7327
           4       0.10      0.72      0.17       329

    accuracy                       

In [12]:
xgb = XGBClassifier()
xgb.fit(X_train, Y_train)
pred_xgb = xgb.predict(X_test)
y_train_pred = xgb.predict(X_train[:30000])

mat_xgb = confusion_matrix(Y_test, pred_xgb)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_xgb}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_xgb))

del pred_xgb, y_train_pred

[20:55:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
confusion matrix :
[[   15     0     0     0]
 [   17 16789  7134   747]
 [    1   981  3990    92]
 [    0     0     0   234]]

confusion matrix :
[[    5    15     5     0]
 [   24 22570 11116  1149]
 [    6  2267  4800   254]
 [    0    80    33   216]]

              precision    recall  f1-score   support

           1       0.45      1.00      0.62        15
           2       0.94      0.68      0.79     24687
           3       0.36      0.79      0.49      5064
           4       0.22      1.00      0.36       234

    accuracy                           0.70     30000
   macro avg       0.49      0.87      0.57     30000
weighted avg       0.84      0.70      0.74     300

In [13]:
xgb_params = {
    'max_depth' : [1,3,5],
    'n_estimators' : [150,200]
}

clf = GridSearchCV(xgb, param_grid=xgb_params,cv=5,scoring='recall')
clf.fit(X_val, Y_val)


xgb = XGBClassifier(**clf.best_params_)
xgb.fit(X_train, Y_train)
pred_xgb = xgb.predict(X_test)
y_train_pred = xgb.predict(X_train[:30000])

mat_xgb = confusion_matrix(Y_test, pred_xgb)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_xgb}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_xgb))

del pred_xgb, y_train_pred

[20:57:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:57:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:57:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:57:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[21:02:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:03:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:03:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:03:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [14]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, Y_train)
pred_lgbm = lgbm.predict(X_test)
y_train_pred = lgbm.predict(X_train[:30000])

mat_lgbm = confusion_matrix(Y_test, pred_lgbm)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_lgbm}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_lgbm))

del pred_lgbm, y_train_pred

confusion matrix :
[[   15     0     0     0]
 [   65 14713  8707  1202]
 [    8  1274  3603   179]
 [    0     0     0   234]]

confusion matrix :
[[    5    15     5     0]
 [   69 20334 12738  1718]
 [   16  2088  4883   340]
 [    1    68    20   240]]

              precision    recall  f1-score   support

           1       0.17      1.00      0.29        15
           2       0.92      0.60      0.72     24687
           3       0.29      0.71      0.41      5064
           4       0.14      1.00      0.25       234

    accuracy                           0.62     30000
   macro avg       0.38      0.83      0.42     30000
weighted avg       0.81      0.62      0.67     30000

              precision    recall  f1-score   support

           1       0.05      0.20      0.09        25
           2       0.90      0.58      0.71     34859
           3       0.28      0.67      0.39      7327
           4       0.10      0.73      0.18       329

    accuracy                       

In [15]:
cat = CatBoostClassifier()
logs = cat.fit(X_train, Y_train)

pred_cat = cat.predict(X_test)
y_train_pred = cat.predict(X_train[:30000])

mat_cat = confusion_matrix(Y_test, pred_cat)
mat_train = confusion_matrix(Y_train[:30000],y_train_pred)

print(f"confusion matrix :\n{mat_train}\n")
print(f"confusion matrix :\n{mat_cat}\n")
print(classification_report(Y_train[:30000],y_train_pred))
print(classification_report(Y_test, pred_cat))

del pred_cat,y_train_pred

Learning rate set to 0.108006
0:	learn: 1.3108891	total: 309ms	remaining: 5m 8s
1:	learn: 1.2539803	total: 478ms	remaining: 3m 58s
2:	learn: 1.2055129	total: 653ms	remaining: 3m 36s
3:	learn: 1.1654407	total: 844ms	remaining: 3m 30s
4:	learn: 1.1296078	total: 1.04s	remaining: 3m 26s
5:	learn: 1.1010909	total: 1.23s	remaining: 3m 23s
6:	learn: 1.0752938	total: 1.4s	remaining: 3m 18s
7:	learn: 1.0494851	total: 1.58s	remaining: 3m 16s
8:	learn: 1.0283224	total: 1.77s	remaining: 3m 15s
9:	learn: 1.0102297	total: 1.97s	remaining: 3m 15s
10:	learn: 0.9931061	total: 2.16s	remaining: 3m 14s
11:	learn: 0.9778522	total: 2.35s	remaining: 3m 13s
12:	learn: 0.9595485	total: 2.55s	remaining: 3m 13s
13:	learn: 0.9399675	total: 2.73s	remaining: 3m 12s
14:	learn: 0.9245379	total: 2.91s	remaining: 3m 11s
15:	learn: 0.9123178	total: 3.11s	remaining: 3m 11s
16:	learn: 0.9022815	total: 3.29s	remaining: 3m 10s
17:	learn: 0.8914774	total: 3.46s	remaining: 3m 8s
18:	learn: 0.8801143	total: 3.63s	remaining: 3m

158:	learn: 0.4898179	total: 30s	remaining: 2m 38s
159:	learn: 0.4888447	total: 30.1s	remaining: 2m 38s
160:	learn: 0.4878773	total: 30.4s	remaining: 2m 38s
161:	learn: 0.4874860	total: 30.5s	remaining: 2m 37s
162:	learn: 0.4867406	total: 30.7s	remaining: 2m 37s
163:	learn: 0.4860112	total: 30.9s	remaining: 2m 37s
164:	learn: 0.4850060	total: 31.1s	remaining: 2m 37s
165:	learn: 0.4839265	total: 31.2s	remaining: 2m 36s
166:	learn: 0.4833618	total: 31.4s	remaining: 2m 36s
167:	learn: 0.4824384	total: 31.6s	remaining: 2m 36s
168:	learn: 0.4818818	total: 31.8s	remaining: 2m 36s
169:	learn: 0.4811409	total: 32s	remaining: 2m 36s
170:	learn: 0.4801409	total: 32.2s	remaining: 2m 35s
171:	learn: 0.4794394	total: 32.3s	remaining: 2m 35s
172:	learn: 0.4781333	total: 32.5s	remaining: 2m 35s
173:	learn: 0.4774710	total: 32.7s	remaining: 2m 35s
174:	learn: 0.4766806	total: 32.9s	remaining: 2m 35s
175:	learn: 0.4754566	total: 33.1s	remaining: 2m 34s
176:	learn: 0.4742968	total: 33.2s	remaining: 2m 3

315:	learn: 0.4048967	total: 59.2s	remaining: 2m 8s
316:	learn: 0.4045746	total: 59.3s	remaining: 2m 7s
317:	learn: 0.4042398	total: 59.5s	remaining: 2m 7s
318:	learn: 0.4039529	total: 59.7s	remaining: 2m 7s
319:	learn: 0.4035183	total: 59.9s	remaining: 2m 7s
320:	learn: 0.4031627	total: 1m	remaining: 2m 7s
321:	learn: 0.4029080	total: 1m	remaining: 2m 6s
322:	learn: 0.4026251	total: 1m	remaining: 2m 6s
323:	learn: 0.4021225	total: 1m	remaining: 2m 6s
324:	learn: 0.4016889	total: 1m	remaining: 2m 6s
325:	learn: 0.4011507	total: 1m	remaining: 2m 6s
326:	learn: 0.4005628	total: 1m 1s	remaining: 2m 5s
327:	learn: 0.4001273	total: 1m 1s	remaining: 2m 5s
328:	learn: 0.3998880	total: 1m 1s	remaining: 2m 5s
329:	learn: 0.3996903	total: 1m 1s	remaining: 2m 5s
330:	learn: 0.3994227	total: 1m 1s	remaining: 2m 5s
331:	learn: 0.3990461	total: 1m 2s	remaining: 2m 4s
332:	learn: 0.3986941	total: 1m 2s	remaining: 2m 4s
333:	learn: 0.3983769	total: 1m 2s	remaining: 2m 4s
334:	learn: 0.3982248	total: 1

470:	learn: 0.3636292	total: 1m 27s	remaining: 1m 38s
471:	learn: 0.3634034	total: 1m 28s	remaining: 1m 38s
472:	learn: 0.3632327	total: 1m 28s	remaining: 1m 38s
473:	learn: 0.3629591	total: 1m 28s	remaining: 1m 38s
474:	learn: 0.3627964	total: 1m 28s	remaining: 1m 37s
475:	learn: 0.3625495	total: 1m 28s	remaining: 1m 37s
476:	learn: 0.3623669	total: 1m 28s	remaining: 1m 37s
477:	learn: 0.3620992	total: 1m 29s	remaining: 1m 37s
478:	learn: 0.3618394	total: 1m 29s	remaining: 1m 37s
479:	learn: 0.3617201	total: 1m 29s	remaining: 1m 36s
480:	learn: 0.3615196	total: 1m 29s	remaining: 1m 36s
481:	learn: 0.3612859	total: 1m 29s	remaining: 1m 36s
482:	learn: 0.3610667	total: 1m 30s	remaining: 1m 36s
483:	learn: 0.3609016	total: 1m 30s	remaining: 1m 36s
484:	learn: 0.3606594	total: 1m 30s	remaining: 1m 36s
485:	learn: 0.3603653	total: 1m 30s	remaining: 1m 35s
486:	learn: 0.3601351	total: 1m 30s	remaining: 1m 35s
487:	learn: 0.3599918	total: 1m 31s	remaining: 1m 35s
488:	learn: 0.3598038	total:

622:	learn: 0.3370524	total: 1m 56s	remaining: 1m 10s
623:	learn: 0.3368504	total: 1m 56s	remaining: 1m 10s
624:	learn: 0.3366714	total: 1m 56s	remaining: 1m 9s
625:	learn: 0.3365315	total: 1m 56s	remaining: 1m 9s
626:	learn: 0.3364630	total: 1m 56s	remaining: 1m 9s
627:	learn: 0.3363016	total: 1m 57s	remaining: 1m 9s
628:	learn: 0.3361810	total: 1m 57s	remaining: 1m 9s
629:	learn: 0.3360672	total: 1m 57s	remaining: 1m 8s
630:	learn: 0.3358871	total: 1m 57s	remaining: 1m 8s
631:	learn: 0.3357710	total: 1m 57s	remaining: 1m 8s
632:	learn: 0.3355859	total: 1m 57s	remaining: 1m 8s
633:	learn: 0.3354984	total: 1m 58s	remaining: 1m 8s
634:	learn: 0.3354015	total: 1m 58s	remaining: 1m 7s
635:	learn: 0.3352637	total: 1m 58s	remaining: 1m 7s
636:	learn: 0.3351858	total: 1m 58s	remaining: 1m 7s
637:	learn: 0.3351126	total: 1m 58s	remaining: 1m 7s
638:	learn: 0.3350119	total: 1m 58s	remaining: 1m 7s
639:	learn: 0.3349112	total: 1m 59s	remaining: 1m 6s
640:	learn: 0.3348009	total: 1m 59s	remainin

779:	learn: 0.3179068	total: 2m 25s	remaining: 41s
780:	learn: 0.3177761	total: 2m 25s	remaining: 40.8s
781:	learn: 0.3176408	total: 2m 25s	remaining: 40.6s
782:	learn: 0.3175402	total: 2m 25s	remaining: 40.4s
783:	learn: 0.3173528	total: 2m 26s	remaining: 40.2s
784:	learn: 0.3172128	total: 2m 26s	remaining: 40.1s
785:	learn: 0.3171352	total: 2m 26s	remaining: 39.9s
786:	learn: 0.3169651	total: 2m 26s	remaining: 39.7s
787:	learn: 0.3167851	total: 2m 26s	remaining: 39.5s
788:	learn: 0.3166765	total: 2m 26s	remaining: 39.3s
789:	learn: 0.3166063	total: 2m 27s	remaining: 39.1s
790:	learn: 0.3165453	total: 2m 27s	remaining: 38.9s
791:	learn: 0.3164126	total: 2m 27s	remaining: 38.7s
792:	learn: 0.3162442	total: 2m 27s	remaining: 38.6s
793:	learn: 0.3161765	total: 2m 27s	remaining: 38.4s
794:	learn: 0.3161164	total: 2m 27s	remaining: 38.2s
795:	learn: 0.3160095	total: 2m 28s	remaining: 38s
796:	learn: 0.3159562	total: 2m 28s	remaining: 37.8s
797:	learn: 0.3158898	total: 2m 28s	remaining: 37.

935:	learn: 0.3025790	total: 2m 53s	remaining: 11.8s
936:	learn: 0.3024465	total: 2m 53s	remaining: 11.7s
937:	learn: 0.3023732	total: 2m 53s	remaining: 11.5s
938:	learn: 0.3023175	total: 2m 53s	remaining: 11.3s
939:	learn: 0.3022295	total: 2m 53s	remaining: 11.1s
940:	learn: 0.3021598	total: 2m 54s	remaining: 10.9s
941:	learn: 0.3020879	total: 2m 54s	remaining: 10.7s
942:	learn: 0.3020123	total: 2m 54s	remaining: 10.6s
943:	learn: 0.3018891	total: 2m 54s	remaining: 10.4s
944:	learn: 0.3018109	total: 2m 54s	remaining: 10.2s
945:	learn: 0.3017553	total: 2m 55s	remaining: 10s
946:	learn: 0.3016739	total: 2m 55s	remaining: 9.81s
947:	learn: 0.3015607	total: 2m 55s	remaining: 9.63s
948:	learn: 0.3014852	total: 2m 55s	remaining: 9.44s
949:	learn: 0.3013771	total: 2m 55s	remaining: 9.26s
950:	learn: 0.3012954	total: 2m 56s	remaining: 9.07s
951:	learn: 0.3011369	total: 2m 56s	remaining: 8.89s
952:	learn: 0.3010754	total: 2m 56s	remaining: 8.71s
953:	learn: 0.3009919	total: 2m 56s	remaining: 8